In [1]:
import mediapy as media
import numpy as np

import glider.visualization as visualization


ImportError: You are running an x86_64 build of Python on an Apple Silicon machine. This is not supported by MuJoCo. Please install and run a native, arm64 build of Python.

# GliderSIM

This is a genetic algorithm to optimise a glider form.

That means we put a population of gliders through a test and see which perform the best.

The best gliders of that generation have offspring and so we improve the performance over time.


## The Gliders

Gliders are a combination of points and faces that make up a polygon. They look like this:

In [ ]:
from glider.vehicle import Vehicle

v = Vehicle(num_vertices=10)

media.show_image(visualization.view_vehicle(v))

In [ ]:
from glider.pipeline import process_genome
from glider.surface import spherical_delaunay_surface

vertices = [list(np.random.random(3)) for _ in range(20)]
vehicle, score = process_genome(
    vertices,
    surface_builder=spherical_delaunay_surface,
)

media.show_image(visualization.view_vehicle(vehicle))
print(f"Pipeline Score: {score}")


## The Test

The test is a drop from 80m.

The glider's score is determined by `abs(x_pos)` at collision. 

The default density is the same as that of an inflated paraglider, hence the slow fall rate.

In [ ]:
import glider.optimization as optimization

test_xml_representation = optimization.drop_test_glider(v)

visualization.render_xml_to_collision(
    test_xml_representation,
    camera_name="fixed",
    show=True
)

The `fitness_func` renders this simulation, and returns the total distance travelled in the x-direction.

In [ ]:
import glider.optimization as optimization

score = optimization.fitness_func(v)

print(f"Glider Score: {score}")

## Getting better gliders

The simplest way to get a better glider, is to try many random gliders. 

In the cell below, we create and test a population of 100 gliders.

In [ ]:
population = [Vehicle(num_vertices=10) for _ in range(100)]

fitnesses = [optimization.fitness_func(v) for v in population]

This is what the top performers looked like:

In [ ]:
ranking = list(zip(population, fitnesses))
ranking.sort(key=lambda x: x[1], reverse=True)  # Sort by highest to lowest fitness

for glider, fitness in ranking[:3]:
    media.show_image(visualization.view_vehicle(glider))
    print(f"Fitness: {fitness}")

Graphing the distribution of the glider performances, we get:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(ranking, columns=["Vehicle", "Fitness"])
df.plot.hist(bins=20)

winning_glider, fitness = ranking[0]

drop_test_xml = optimization.drop_test_glider(winning_glider)

visualization.render_xml_to_collision(
    drop_test_xml,
    camera_name="fixed",
    show=True
)


In [ ]:
winning_glider, fitness = ranking[0]

drop_test_xml = optimization.drop_test_glider(winning_glider)

visualization.render_xml_to_collision(
    drop_test_xml,
    camera_name="fixed",
    show=True
)


import time
from io import BytesIO

import numpy as np
from PIL import Image

from glider.optimization import iterate_population, fitness_func, NUM_GENES

NUM_GENERATIONS = 300
POPULATION_SIZE = 100

t0 = time.perf_counter()
input_population = [Vehicle(num_vertices=NUM_GENES) for _ in range(POPULATION_SIZE)]

best_glider_frames = []
population_performance_frames = []
for i in range(NUM_GENERATIONS):
    # Take an input population, and select vehicles to survive based on 
    # survival_weight and cloning_weight

    # Create a new generation by cloning and mutating the surviving vehicles
    # to achieve the same population size


    input_ranking, new_population = iterate_population(
        input_population=input_population,
        survival_weight = 0.1,
        cloning_weight = 0.1,
    )

    if i % 10 == 0:
        print("Iteration:", i)

        winning_vehicle = input_ranking[0][0]
        print("Fitness:", fitness_func(winning_vehicle))
    
    best_glider_frames.append(visualization.view_vehicle(winning_vehicle))
    population_performance_frames.append(visualization.graph_population(input_ranking))

    input_population = new_population

t1 = time.perf_counter()
ga_time = t1 - t0

print(f"Time taken: {ga_time:.2f} seconds")


In [ ]:
import time
from io import BytesIO

import numpy as np
from PIL import Image

from glider.optimization import iterate_population, fitness_func, NUM_GENES

NUM_GENERATIONS = 300
POPULATION_SIZE = 100

t0 = time.perf_counter()
input_population = [Vehicle(num_vertices=NUM_GENES) for _ in range(POPULATION_SIZE)]

best_glider_frames = []
population_performance_frames = []
for i in range(NUM_GENERATIONS):
    # Take an input population, and select vehicles to survive based on 
    # survival_weight and cloning_weight

    # Create a new generation by cloning and mutating the surviving vehicles
    # to achieve the same population size


    input_ranking, new_population = iterate_population(
        input_population=input_population,
        survival_weight = 0.1,
        cloning_weight = 0.1,
    )

    if i % 10 == 0:
        print("Iteration:", i)

        winning_vehicle = input_ranking[0][0]
        print("Fitness:", fitness_func(winning_vehicle))
    
    best_glider_frames.append(visualization.view_vehicle(winning_vehicle))
    population_performance_frames.append(visualization.graph_population(input_ranking))

    input_population = new_population

t1 = time.perf_counter()
ga_time = t1 - t0

print(f"Time taken: {ga_time:.2f} seconds")


In [ ]:
media.show_video(population_performance_frames, fps=10)

In [ ]:
media.show_video(best_glider_frames, fps=3)
best_glider_frames[0].shape

In [ ]:
from glider.optimization import evaluate_population
# Test the best glider of a 300 * 100 population

t2 = time.perf_counter()

mega_population = [Vehicle(num_vertices=NUM_GENES) for _ in range(NUM_GENERATIONS * POPULATION_SIZE)]
ranking = evaluate_population(mega_population)

best_random_glider, fitness = ranking[0]

t3 = time.perf_counter()
brute_force_time = t3 - t2

print("Best fitness from mega_population:", fitness)
print("Best fitness from genetic algorithm:", fitness_func(winning_vehicle))


In [ ]:
ga_time = 8*60 + 41
brute_force_time = 6*60 + 40

print("Percentage improvement:", (fitness_func(winning_vehicle) - fitness) / fitness * 100)

print(f"Time taken: {brute_force_time:.2f} seconds")

ga_improvement_per_second = fitness_func(winning_vehicle) / ga_time
brute_force_improvement_per_second = fitness / brute_force_time

print("Genetic algorithm improvement per second:", ga_improvement_per_second)
print("Brute force improvement per second:", brute_force_improvement_per_second)

time_efficiency_gain = (ga_improvement_per_second - brute_force_improvement_per_second) / brute_force_improvement_per_second * 100
print("Time Efficiency Gain:", time_efficiency_gain)

test_xml_representation = optimization.drop_test_glider(winning_vehicle)

visualization.render_xml_to_collision(
    test_xml_representation,
    camera_name="fixed",
    show=True
)


In [ ]:
test_xml_representation = optimization.drop_test_glider(winning_vehicle)

visualization.render_xml_to_collision(
    test_xml_representation,
    camera_name="fixed",
    show=True
)


In [ ]:
print(test_xml_representation)